The base code of this notebook was taken from https://www.kaggle.com/piantic/no-tta-cassava-resnext50-32x4d-inference-lb0-903

The idea for dataset class was derived from https://www.kaggle.com/ihelon/brain-tumor-eda-with-animations-and-modeling

# Directory settings

In [ ]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    
TRAIN_PATH = '../input/rsna-miccai-png/train'
TEST_PATH = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/test'

# CFG

In [ ]:
# ====================================================
# CFG
# ====================================================
class CFG:
    debug=False
    num_workers=8
    model_name='efficientnet_b3'
    size=512
    batch_size=4
    seed=42
    target_size=2
    target_col='MGMT_value'
    n_fold=5
    trn_fold=[0,1,2,3,4]
    inference=True

# Imports

In [ ]:
# ====================================================
# Imports
# ====================================================
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import timm

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Reproducibility

In [ ]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

# Data Loading

In [ ]:
test = os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test')
test = pd.DataFrame({'BraTS21ID' : test})
test['BraTS21ID'] = test['BraTS21ID'].astype(int)
test.head()

# Dataset

In [ ]:
# ====================================================
# Dataset
# ====================================================
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['BraTS21ID'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        root = f'{TEST_PATH}/{str(self.file_names[idx]).zfill(5)}/'
        com = []
        for typ in ['FLAIR', 'T1w', 'T1wCE', 'T2w']:
            paths = os.listdir(root + typ)
            rnd = random.sample(paths, min(10, len(paths)))
            typ_imgs = []
            for f in rnd:
                file_path = f'{root}{typ}/{f}'
                dicom = pydicom.read_file(file_path)
                data = apply_voi_lut(dicom.pixel_array, dicom)
                if dicom.PhotometricInterpretation == "MONOCHROME1":
                    data = np.amax(data) - data
                data = data - np.min(data)
                data = data / np.max(data)
                image = (data * 255).astype(np.uint8)
                typ_imgs.append(cv2.resize(image, (CFG.size, CFG.size)))
            com.append(np.mean(typ_imgs, axis = 0))
        image = np.array(com).transpose((1,2,0)) / 255
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            image = image.float()
        return image

# Transforms

In [ ]:
# ====================================================
# Transforms
# ====================================================
def get_transforms(*, data):
    if data == 'valid':
        return A.Compose([
            ToTensorV2(),
        ])

# MODEL

In [ ]:
# ====================================================
# MODEL
# ====================================================
class CustomEfficientNet(nn.Module):
    def __init__(self, model_name=CFG.model_name, pretrained=False):
        super().__init__()
        self.conv = nn.Conv2d(4,3,1)
        self.model = timm.create_model(CFG.model_name, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, CFG.target_size)

    def forward(self, x):
        x = self.conv(x)
        x = self.model(x)
        return x

# Helper functions

In [ ]:
# ====================================================
# Helper functions
# ====================================================
def load_state_eff(model_path):
    state_dict = torch.load(model_path)['model']
    return state_dict

def inference(model_eff, states, test_loader, device):
    model_eff.to(device)
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        x=1
        for state in states:
            model_eff.load_state_dict(state)
            model_eff.eval()
            with torch.no_grad():
                y_preds = model_eff(images)
            avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
            x+=1
            
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

# inference

In [ ]:
model_eff = CustomEfficientNet(CFG.model_name, pretrained=False)

states = [load_state_eff('../input/rsna-training/efficientnet_b3_fold0_best.pth'),
          load_state_eff('../input/rsna-training/efficientnet_b3_fold1_best.pth'),
          load_state_eff('../input/rsna-training/efficientnet_b3_fold2_best.pth'),
          load_state_eff('../input/rsna-training/efficientnet_b3_fold3_best.pth'),
          load_state_eff('../input/rsna-training/efficientnet_b3_fold4_best.pth'),
]

In [ ]:
# ====================================================
# inference
# ====================================================

test_dataset = TestDataset(test, transform=get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True)
predictions = inference(model_eff, states, test_loader, device)

# submission
test['MGMT_value'] = predictions[:,1]
test[['BraTS21ID', 'MGMT_value']].to_csv(OUTPUT_DIR+'submission.csv', index=False)
test.head()